在代码注释中，`P(origin=model)`、`P(origin=noise)` 和 `P(noise=sample)` 分别表示以下含义：

### 1. **`P(origin=model)`**
- **含义**：表示输入特征 `x` 实际来自正样本的概率，即模型认为该样本来自原始数据分布的概率。
- **解释**：在对比学习中，我们希望区分正样本（即原始数据的真实样本）和噪声样本（或负样本，通常为随机采样的其他样本）。`P(origin=model)` 是模型预测输入 `x` 是正样本的概率。这个概率通过计算正样本的相似度与所有样本相似度的比值来表示。

### 2. **`P(origin=noise)`**
- **含义**：表示输入特征 `x` 实际来自噪声样本的概率，即模型认为该样本来自噪声分布的概率。
- **解释**：噪声样本是从所有可能的样本中随机选择的，并非原始数据分布中的样本。`P(origin=noise)` 是模型预测输入 `x` 是噪声样本的概率。这个概率是通过计算负样本的相似度与所有样本相似度的比值来表示的。

### 3. **`P(noise=sample)`**
- **含义**：表示一个样本被选作噪声样本的概率。
- **解释**：`P(noise=sample)` 是先验概率，表示某个样本被当作噪声样本的概率。对于每一个样本，这个概率在训练集中是均匀分布的，因此为 `1 / nLem`，其中 `nLem` 是训练集中的样本总数。

### 代码中这些概率的作用

1. **`P(origin=model)`**:
   - 该概率用于判断当前输入特征 `x` 是来自于正样本的可能性，是模型的输出（第 0 列）的概率。
   - 在实现中，通过公式 `P(origin=model) = Pmt / (Pmt + K * Pnt)` 计算，其中 `Pmt` 是模型输出的相似度分数，`K * Pnt` 是噪声样本的贡献。

2. **`P(origin=noise)`**:
   - 该概率用于判断当前输入特征 `x` 是来自于噪声样本的可能性。
   - 在实现中，通过公式 `P(origin=noise) = K * Pns / (Pms + K * Pns)` 计算，其中 `Pms` 是噪声样本的相似度分数，`K * Pns` 是噪声样本的先验概率。

3. **`P(noise=sample)`**:
   - 这是一个固定的先验概率，表示任何一个样本被作为噪声样本的概率。因为所有样本均匀分布，这个概率为 `1 / nLem`，其中 `nLem` 是所有样本的数量。

### 总结

- **`P(origin=model)`** 是模型预测输入特征 `x` 为正样本的概率。
- **`P(origin=noise)`** 是模型预测输入特征 `x` 为噪声样本的概率。
- **`P(noise=sample)`** 是一个样本被作为噪声样本的先验概率。

In [ ]:
import torch
from torch import nn

eps = 1e-7

class NCECriterion(nn.Module):

    def __init__(self, nLem):
        super(NCECriterion, self).__init__()
        self.nLem = nLem # nLem 表示 memory bank 的大小，即样本的总数

    def forward(self, x, targets):
        # x shape: [batchSize, K+1]
        # targets shape: [batchSize]
        # K is the number of noise samples
        batchSize = x.size(0)
        K = x.size(1)-1 # K 是负样本的数量，x 的第二维是 K+1（包括一个正样本和 K 个负样本）
        Pnt = 1 / float(self.nLem)  # P(origin=noise)
        Pns = 1 / float(self.nLem)  # P(noise=sample)
        
        # eq 5.1 : P(origin=model) = Pmt / (Pmt + k*Pnt) 
        Pmt = x.select(1,0)  # 1st column is the model output
        Pmt_div = Pmt.add(K * Pnt + eps)
        lnPmt = torch.div(Pmt, Pmt_div)
        
        # eq 5.2 : P(origin=noise) = k*Pns / (Pms + k*Pns)
        Pon_div = x.narrow(1,1,K).add(K * Pns + eps)  # 2nd to last column are noise samples
        Pon = Pon_div.clone().fill_(K * Pns)
        lnPon = torch.div(Pon, Pon_div)
     
        # equation 6 in ref. A
        lnPmt.log_()
        lnPon.log_()
        
        lnPmtsum = lnPmt.sum(0)
        lnPonsum = lnPon.view(-1, 1).sum(0)
        
        loss = - (lnPmtsum + lnPonsum) / batchSize

        return loss

In [1]:
from PIL import Image
import torchvision.datasets as datasets
import torch.utils.data as data

In [2]:
class CIFAR10Instance(datasets.CIFAR10):
    """CIFAR10Instance Dataset.
    """
    def __getitem__(self, index):
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index

In [3]:
import torch
import torchvision.transforms as transforms

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(size=32, scale=(0.2,1.)),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
    transforms.RandomGrayscale(p=0.2),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = CIFAR10Instance(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = CIFAR10Instance(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
for batch_idx, (inputs, targets, indexes) in enumerate(trainloader):
    inputs, targets, indexes = inputs.to(device), targets.to(device), indexes.to(device)
    print(batch_idx, img.shape, target.shape, index.shape)
    print(img, target, index)
    break

In [ ]:
for temp in trainloader:
    print(len(temp))
    break

In [ ]:
for img,target,index in trainloader:
    print(img.shape, target.shape, index.shape)
    print(img, target, index)
    break